In [27]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch
import yaml

import os
import random
from ultralytics.utils.plotting import plot_labels
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

import visualization_utils as visutils

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40339MiB)
Setup complete ✅ (36 CPUs, 376.5 GB RAM, 0.6/188.3 GB disk)


/vast/palmer/home.grace/eec42/BirdDetector/src
/vast/palmer/home.grace/eec42/BirdDetector/src/data_preprocessing


In [28]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
model = YOLO('yolov8m.pt')

In [4]:
# TRAIN the model on our dataset (data.yml config file) 
model_name = 'pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch'

results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=120,
   batch=32,
   #cos_lr=True,
   #dropout=0.3,
   #optimizer='Adam',
   patience=50,
   device=0,
   verbose=True,
   #lr0=0.001,
   #lrf=0.0001,
   degrees=90, fliplr=0.5, flipud=0.5, # augmentation parameters
   name=model_name)

New https://pypi.org/project/ultralytics/8.0.216 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=120, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width

  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384, 576, 3, 2]              
  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576,

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/train
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/train


train: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/train/labels... 5183 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5183/5183 [00:01<00:00, 3051.01it/s]
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_0.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_2.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_2.0_0.0_448_448.jpg: 8 duplicate labels remov

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/val
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/val


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/val/labels... 579 images, 0 backgrounds, 0 corrupt: 100%|██████████| 579/579 [00:00<00:00, 2930.82it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/val/labels.cache
/home/eec42/.conda/envs/pdm/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 10, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Plotting labels to runs/detect/pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch/labels.jpg... 
optimizer: 'optimizer=auto' found, ignori

## Evaluation

In [5]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/labels... 1010 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1010/1010 [00:00<00:00, 1947.87it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:14<00:00,  2.24it/s]
                   all       1010      18771      0.629      0.627      0.561      0.225
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 2.2ms postprocess per image
Saving runs/detect/pfeifer_penguins_poland_yolov8m_120epoch2/predictions.json...
Results saved to runs/detect/pfeifer_penguins_poland_yolov8m_120epoch2


In [6]:
print(metrics.box.map)    # map50-95
print(metrics.box.map50)  # map50
#metrics.box.map75  # map75
#metrics.box.maps   # a list contains map50-95 of each category

0.22511776009070622
0.5610286115150127


In [5]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True, iou=0.1, max_det=600)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/test/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/test/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/test/labels... 1407 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1407/1407 [00:00<00:00, 3496.19it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_palmyra_mckellar_no_background/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:14<00:00,  3.13it/s]
                   all       1407      19382      0.742      0.797      0.779      0.292
Speed: 0.1ms preprocess, 5.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Saving runs/detect/pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch2/predictions.json...
Results saved to runs/detect/pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch2


In [6]:
print(metrics.box.map)
print(metrics.box.map50)    # map50-95

0.29207870339463005
0.7788214356152334


## Evaluation per dataset

In [5]:
datasets = ['global_birds_pfeifer', 'global_birds_penguins', 'global_birds_poland']

In [6]:
for dataset in datasets:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    metrics = model.val(split='test', save_json=True, iou=0.1, max_det=600)
    print(metrics.box.map50)

# Change test folder to original name
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
data['test'] = 'test/images/'

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)


Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_pfeifer/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_pfeifer/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_pfeifer/labels.cache... 677 images, 0 backgrounds, 0 corrupt: 100%|██████████| 677/677 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:06<00:00,  6.22it/s]
                   all        677      14165      0.817      0.798      0.805      0.334
Speed: 0.2ms preprocess, 5.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Saving runs/detect/val4/predictions.json...
Results saved to runs/detect/val4
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)


0.8054123993792808
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_penguins/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_penguins/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_penguins/labels... 154 images, 0 backgrounds, 0 corrupt: 100%|██████████| 154/154 [00:00<00:00, 2194.41it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_penguins/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.14it/s]
                   all        154       3806      0.225      0.281     0.0923       0.02
Speed: 0.5ms preprocess, 5.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Saving runs/detect/val5/predictions.json...
Results saved to runs/detect/val5
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)


0.09225974587433548
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_poland/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_poland/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/global_birds_poland/labels.cache... 179 images, 0 backgrounds, 0 corrupt: 100%|██████████| 179/179 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:42<00:00,  3.52s/it]
                   all        179        800      0.471      0.525      0.409      0.144
Speed: 0.4ms preprocess, 5.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Saving runs/detect/val6/predictions.json...
Results saved to runs/detect/val6


0.40920486155985425


## Visualization

In [5]:
#model_name = 'pfeifer_penguins_yolov8m_120epoch'
#model = YOLO('runs/detect/' + model_name + '/weights/best.pt')#


In [7]:
datasets = ['global_birds_pfeifer', 'global_birds_penguins', 'global_birds_poland'] #, 'global_birds_palmyra', 'global_birds_mckellar']

fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

In [8]:
# Select randomly 10 images from the test dataset
selected_img = []
for subdataset in datasets:
    selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=12))

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.2, 
        iou = 0.1,
        show=False,
        save=False
    )


0: 640x640 97 birds, 1: 640x640 2 birds, 2: 640x640 4 birds, 3: 640x640 3 birds, 4: 640x640 105 birds, 5: 640x640 1 bird, 6: 640x640 41 birds, 7: 640x640 17 birds, 8: 640x640 37 birds, 9: 640x640 60 birds, 10: 640x640 10 birds, 11: 640x640 9 birds, 12: 640x640 1 bird, 13: 640x640 (no detections), 14: 640x640 1 bird, 15: 640x640 10 birds, 16: 640x640 1 bird, 17: 640x640 10 birds, 18: 640x640 (no detections), 19: 640x640 1 bird, 20: 640x640 30 birds, 21: 640x640 37 birds, 22: 640x640 1 bird, 23: 640x640 14 birds, 24: 640x640 22 birds, 25: 640x640 (no detections), 26: 640x640 1 bird, 27: 640x640 1 bird, 28: 640x640 2 birds, 29: 640x640 7 birds, 30: 640x640 1 bird, 31: 640x640 1 bird, 32: 640x640 2 birds, 33: 640x640 1 bird, 34: 640x640 2 birds, 35: 640x640 2 birds, 36: 640x640 (no detections), 37: 640x640 (no detections), 38: 640x640 (no detections), 39: 640x640 (no detections), 40: 640x640 (no detections), 41: 640x640 (no detections), 42: 640x640 (no detections), 43: 640x640 (no detecti

In [9]:
for img, result in zip(selected_img, results):

    detection_boxes = []
    save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
    for detect in range(len(result.boxes.cls)):
        det = {}
        det['conf'] = result.boxes.conf[detect].cpu()
        det['category'] = result.boxes.cls[detect].cpu()
        coords = result.boxes.xywhn[detect].cpu()
        det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
        detection_boxes.append(det)
        
    im_path = os.path.join(img_path + 'images/', img)
    visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['Red'])

    selected_label = img_path + 'labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
    detection_boxes = []
    df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
    for irow, row in df.iterrows():  
        det = {}
        det['conf'] = None
        det['category'] = row[0]
        det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
        detection_boxes.append(det)
    
    # Draw annotations
    save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
    visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['SpringGreen'])


## **Customed evaluation**

In [5]:

model_name = 'pfeifer_penguins_poland_yolov8m_120epoch'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
#metrics = model.val(split='test', save_json=True, conf=0.2, iou=0.1, max_det=600)

In [29]:
# Compare predictions from model.val with predictions form model.predict
import json

model_name = 'pfeifer_penguins_poland_yolov8m_120epoch_iou0.1_conf0.2'
f = open('runs/detect/' + model_name + '/predictions.json')
eval_pred = json.load(f)

In [30]:
# predictions from model.predict
model_name = 'pfeifer_penguins_poland_yolov8m_120epoch'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'
test_imgs = os.listdir(img_path + 'images/')

pred_pred = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = img_path + 'images/',
        conf = 0.2, 
        iou = 0.1,
        show=False,
        save=False
    )



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1010 /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/images/global_birds_penguins_cape_wallace_survey_8_452_patch_0.0_0.0_480_480.jpg: 640x640 52 birds, 7.5ms
image 2/1010 /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/images/global_birds_penguins_cape_wallace_survey_8_452_patch_0.0_20.0_480_480.jpg: 640x640 51 birds, 7.5ms
image 3/1010 /gpfs/gibbs/project/jetz/e

In [31]:
print(len(pred_pred[0].boxes.conf))
conf_sup = [conf for conf in pred_pred[0].boxes.conf if conf>0.2]
print(len(conf_sup))

52
52


In [40]:
def intersection_over_union(gt_box, pred_box):
    #print(gt_box)
    #print(pred_box)
    inter_box_top_left = [max(gt_box[0]-gt_box[2]/2, pred_box[0]-pred_box[2]/2), max(gt_box[1]-gt_box[3]/2, pred_box[1]-pred_box[3]/2)]
    inter_box_bottom_right = [min(gt_box[0]+gt_box[2]/2, pred_box[0]+pred_box[2]/2), min(gt_box[1]+gt_box[3]/2, pred_box[1]+pred_box[3]/2)]

    inter_box_w = inter_box_bottom_right[0] - inter_box_top_left[0]
    inter_box_h = inter_box_bottom_right[1] - inter_box_top_left[1]

    intersection = inter_box_w * inter_box_h
    union = gt_box[2] * gt_box[3] + pred_box[2] * pred_box[3] - intersection
    
    iou = intersection / union

    return iou, intersection, union

In [46]:
from pathlib import Path
import numpy as np
from itertools import product

TP = 0 # predicted bird and actual bird
FN = 0 # not predicted bird and actual bird
FP = 0 # predicted bird and not actual bird
TN = 0 # not predicted bird and no bird

iou_threshold = 0.1

for pred in pred_pred:

    img_name = os.path.basename(pred.path).split('.jpg')[0]
    img_size = pred.orig_shape
    pred_conf = pred.boxes.conf
    pred_coords = pred.boxes.xywh.cpu().numpy()

    label_path = os.path.join(img_path + 'labels/', img_name + '.txt')
    label_coords = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            coords = np.array(list(map(float, line.split()[1:])))
            label_coords.append(coords*img_size[0])


    gt = ['unpredicted' for i in range(len(label_coords))]
    predictions = ['unverified' for i in range(len(pred_coords))]

    for i, pred_coord in enumerate(pred_coords):
        iou = False

        for j, label_coord in enumerate(label_coords):
            iou_val, inter, union = intersection_over_union(label_coord, pred_coord)
            #print(iou_val, inter, union)
            if iou_val >= iou_threshold:
                gt[j] = 'predicted'
                predictions[i] = 'verified'


    TP += gt.count('predicted') 
    FN += gt.count('unpredicted')
    FP += predictions.count('unverified')

    print("Nb of birds in img: ", len(gt))
    print(gt.count('predicted'), "  are predicted ;")
    print(gt.count('unpredicted'), " are not predicted")
    print(gt.count('predicted')+gt.count('unpredicted'))


Nb of birds in img:  57
57   are predicted ;
0  are not predicted
57
Nb of birds in img:  56
56   are predicted ;
0  are not predicted
56
Nb of birds in img:  64
62   are predicted ;
2  are not predicted
64
Nb of birds in img:  63
63   are predicted ;
0  are not predicted
63
Nb of birds in img:  27
27   are predicted ;
0  are not predicted
27
Nb of birds in img:  30
30   are predicted ;
0  are not predicted
30
Nb of birds in img:  27
27   are predicted ;
0  are not predicted
27
Nb of birds in img:  30
30   are predicted ;
0  are not predicted
30
Nb of birds in img:  93
93   are predicted ;
0  are not predicted
93
Nb of birds in img:  95
95   are predicted ;
0  are not predicted
95
Nb of birds in img:  91
91   are predicted ;
0  are not predicted
91
Nb of birds in img:  94
94   are predicted ;
0  are not predicted
94
Nb of birds in img:  74
74   are predicted ;
0  are not predicted
74
Nb of birds in img:  73
73   are predicted ;
0  are not predicted
73
Nb of birds in img:  76
76   are p

In [48]:
print(TP)
print(FN)
print(FP)

18544
227
1046


In [30]:
# FOR 1 image in particular
img_id = 'global_birds_poland_4_2_patch_0.0_560.0_640_640'

eval_pred_patch4_2 = [item for item in eval_pred if item.get("image_id") == img_id]
eval_pred_patch4_2
#len(eval_pred_patch4_2)

eval_pred_patch4_2_score = [item for item in eval_pred_patch4_2 if item.get("score") > 0.2]
eval_pred_patch4_2_score
#len(eval_pred_patch4_2_score)

[{'image_id': 'global_birds_poland_4_2_patch_0.0_560.0_640_640',
  'category_id': 0,
  'bbox': [256.019, 309.64, 53.23, 54.268],
  'score': 0.40914},
 {'image_id': 'global_birds_poland_4_2_patch_0.0_560.0_640_640',
  'category_id': 0,
  'bbox': [413.734, 206.72, 53.171, 55.571],
  'score': 0.40152},
 {'image_id': 'global_birds_poland_4_2_patch_0.0_560.0_640_640',
  'category_id': 0,
  'bbox': [487.386, 82.948, 51.61, 51.308],
  'score': 0.32165},
 {'image_id': 'global_birds_poland_4_2_patch_0.0_560.0_640_640',
  'category_id': 0,
  'bbox': [19.722, 238.239, 55.96, 54.74],
  'score': 0.28464},
 {'image_id': 'global_birds_poland_4_2_patch_0.0_560.0_640_640',
  'category_id': 0,
  'bbox': [490.037, 367.068, 59.503, 60.548],
  'score': 0.28315},
 {'image_id': 'global_birds_poland_4_2_patch_0.0_560.0_640_640',
  'category_id': 0,
  'bbox': [419.182, 551.554, 59.423, 57.368],
  'score': 0.26602},
 {'image_id': 'global_birds_poland_4_2_patch_0.0_560.0_640_640',
  'category_id': 0,
  'bbox': [

In [5]:
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'


pred_pred = model.predict(
        source = img_path,
        conf = 0.2, 
        iou = 0.1,
        max_det = 600,
        show=False,
        save=False
    )



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1010 /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/images/global_birds_penguins_cape_wallace_survey_8_452_patch_0.0_0.0_480_480.jpg: 640x640 52 birds, 5.9ms
image 2/1010 /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_penguins_poland_no_background/test/images/global_birds_penguins_cape_wallace_survey_8_452_patch_0.0_20.0_480_480.jpg: 640x640 51 birds, 6.1ms
image 3/1010 /gpfs/gibbs/project/jetz/e

In [53]:
datasets = ['global_birds_pfeifer', 'global_birds_penguins', 'global_birds_poland'] #, 'global_birds_palmyra', 'global_birds_mckellar']
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'


# Select randomly 10 images from the test dataset
selected_img = []
for subdataset in datasets:
    selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=12))

selected_img = ['global_birds_poland_4_2_patch_0.0_0.0_640_640.jpg', 'global_birds_poland_4_2_patch_0.0_560.0_640_640.jpg', 'global_birds_poland_4_2_patch_560.0_0.0_640_640.jpg', 'global_birds_poland_4_2_patch_560.0_560.0_640_640.jpg']
selected_img = ['global_birds_poland_4_2_patch_0.0_560.0_640_640.jpg']

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.2, 
        iou = 0.1,
        show=False,
        save=False
    )


0: 640x640 16 birds, 6.0ms
Speed: 17.7ms preprocess, 6.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [54]:
print(len(results[0].boxes.cls))
results[0].boxes.data

16


tensor([[1.1024e+02, 2.8494e+02, 1.8230e+02, 3.5782e+02, 5.1666e-01, 0.0000e+00],
        [4.1511e+02, 5.4432e+02, 4.8653e+02, 6.1637e+02, 5.0651e-01, 0.0000e+00],
        [4.8067e+02, 7.2634e+01, 5.4847e+02, 1.4450e+02, 5.0494e-01, 0.0000e+00],
        [2.4952e+02, 3.0270e+02, 3.1880e+02, 3.7176e+02, 4.9934e-01, 0.0000e+00],
        [5.2260e+01, 5.9868e+02, 1.1351e+02, 6.3955e+02, 4.9064e-01, 0.0000e+00],
        [4.0855e+02, 1.9971e+02, 4.7325e+02, 2.6613e+02, 4.8802e-01, 0.0000e+00],
        [1.4553e+01, 2.2962e+02, 8.7181e+01, 3.0234e+02, 4.4939e-01, 0.0000e+00],
        [4.8738e+02, 3.6104e+02, 5.5534e+02, 4.3225e+02, 4.3264e-01, 0.0000e+00],
        [4.5526e+02, 4.1900e+02, 5.2847e+02, 4.8531e+02, 4.0561e-01, 0.0000e+00],
        [1.7939e+02, 4.2748e+02, 2.4697e+02, 4.9580e+02, 3.9571e-01, 0.0000e+00],
        [5.0011e+02, 0.0000e+00, 5.5640e+02, 4.2594e+01, 3.4250e-01, 0.0000e+00],
        [0.0000e+00, 4.4071e+02, 4.3981e+01, 5.1278e+02, 3.3352e-01, 0.0000e+00],
        [3.8572e

In [12]:
for img, result in zip(selected_img, results):

    detection_boxes = []
    save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/predictions/pred_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
    for detect in range(len(result.boxes.cls)):
        det = {}
        det['conf'] = result.boxes.conf[detect].cpu()
        det['category'] = result.boxes.cls[detect].cpu()
        coords = result.boxes.xywhn[detect].cpu()
        det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
        detection_boxes.append(det)
        
    im_path = os.path.join(img_path + 'images/', img)
    visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['Red'])

    selected_label = img_path + 'labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
    detection_boxes = []
    df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
    for irow, row in df.iterrows():  
        det = {}
        det['conf'] = None
        det['category'] = row[0]
        det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
        detection_boxes.append(det)
    
    # Draw annotations
    save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/predictions/pred_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
    visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['SpringGreen'])
